# Flights Data Exploration Challenge

In this challenge, you'll explore a real-world dataset containing flights data from the US Department of Transportation.

Let's start by loading and viewing the data.

In [2]:
import pandas as pd

df_flights = pd.read_csv('flights.csv')
df_flights.head()

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,OriginAirportName,OriginCity,OriginState,DestAirportID,DestAirportName,DestCity,DestState,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,9,16,1,DL,15304,Tampa International,Tampa,FL,12478,John F. Kennedy International,New York,NY,1539,4,0.0,1824,13,0,0
1,2013,9,23,1,WN,14122,Pittsburgh International,Pittsburgh,PA,13232,Chicago Midway International,Chicago,IL,710,3,0.0,740,22,1,0
2,2013,9,7,6,AS,14747,Seattle/Tacoma International,Seattle,WA,11278,Ronald Reagan Washington National,Washington,DC,810,-3,0.0,1614,-7,0,0
3,2013,7,22,1,OO,13930,Chicago O'Hare International,Chicago,IL,11042,Cleveland-Hopkins International,Cleveland,OH,804,35,1.0,1027,33,1,0
4,2013,5,16,4,DL,13931,Norfolk International,Norfolk,VA,10397,Hartsfield-Jackson Atlanta International,Atlanta,GA,545,-1,0.0,728,-9,0,0


The dataset contains observations of US domestic flights in 2013, and consists of the following fields:

- **Year**: The year of the flight (all records are from 2013)
- **Month**: The month of the flight
- **DayofMonth**: The day of the month on which the flight departed
- **DayOfWeek**: The day of the week on which the flight departed - from 1 (Monday) to 7 (Sunday)
- **Carrier**: The two-letter abbreviation for the airline.
- **OriginAirportID**: A unique numeric identifier for the departure aiport
- **OriginAirportName**: The full name of the departure airport
- **OriginCity**: The departure airport city
- **OriginState**: The departure airport state
- **DestAirportID**: A unique numeric identifier for the destination aiport
- **DestAirportName**: The full name of the destination airport
- **DestCity**: The destination airport city
- **DestState**: The destination airport state
- **CRSDepTime**: The scheduled departure time
- **DepDelay**: The number of minutes departure was delayed (flight that left ahead of schedule have a negative value)
- **DelDelay15**: A binary indicator that departure was delayed by more than 15 minutes (and therefore considered "late")
- **CRSArrTime**: The scheduled arrival time
- **ArrDelay**: The number of minutes arrival was delayed (flight that arrived ahead of schedule have a negative value)
- **ArrDelay15**: A binary indicator that arrival was delayed by more than 15 minutes (and therefore considered "late")
- **Cancelled**: A binary indicator that the flight was cancelled

Your challenge is to explore the flight data to analyze possible factors that affect delays in departure or arrival of a flight.

1. Start by cleaning the data.
    - Identify any null or missing data, and impute appropriate replacement values.
    - Identify and eliminate any outliers in the **DepDelay** and **ArrDelay** columns.
2. Explore the cleaned data.
    - View summary statistics for the numeric fields in the dataset.
    - Determine the distribution of the **DepDelay** and **ArrDelay** columns.
    - Use statistics, aggregate functions, and visualizations to answer the following questions:
        - *What are the average (mean) departure and arrival delays?*
        - *How do the carriers compare in terms of arrival delay performance?*
        - *Is there a noticable difference in arrival delays for different days of the week?*
        - *Which departure airport has the highest average departure delay?*
        - *Do **late** departures tend to result in longer arrival delays than on-time departures?*
        - *Which route (from origin airport to destination airport) has the most **late** arrivals?*
        - *Which route has the highest average arrival delay?*
        
Add markdown and code cells as required to create your solution.

> **Note**: There is no single "correct" solution. A sample solution is provided in [01 - Flights Challenge.ipynb](01%20-%20Flights%20Solution.ipynb).

### Adding libraries

In [41]:
import plotly as px
import sklearn.impute

### Cleaning data

In [20]:
df_flights.shape

(271940, 20)

In [22]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271940 entries, 0 to 271939
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Year               271940 non-null  int64  
 1   Month              271940 non-null  int64  
 2   DayofMonth         271940 non-null  int64  
 3   DayOfWeek          271940 non-null  int64  
 4   Carrier            271940 non-null  object 
 5   OriginAirportID    271940 non-null  int64  
 6   OriginAirportName  271940 non-null  object 
 7   OriginCity         271940 non-null  object 
 8   OriginState        271940 non-null  object 
 9   DestAirportID      271940 non-null  int64  
 10  DestAirportName    271940 non-null  object 
 11  DestCity           271940 non-null  object 
 12  DestState          271940 non-null  object 
 13  CRSDepTime         271940 non-null  int64  
 14  DepDelay           271940 non-null  int64  
 15  DepDel15           269179 non-null  float64
 16  CR

In [31]:
df_flights.describe().round(2)

,Year,Month,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
count,271940.0,271940.00,271940.00,271940.00,271940.0,271940.00,271940.00,271940.00,269179.0,271940.00,271940.00,271940.00,271940.00
mean,2013.0,6.98,15.81,3.90,12744.9,12739.14,1325.50,10.35,0.2,1505.48,6.50,0.22,0.01
std,0.0,1.98,8.80,1.99,1501.2,1502.57,470.75,35.67,0.4,493.20,38.23,0.41,0.10
min,2013.0,4.00,1.00,1.00,10140.0,10140.00,1.00,-63.00,0.0,1.00,-75.00,0.00,0.00
25%,2013.0,5.00,8.00,2.00,11292.0,11292.00,920.00,-4.00,0.0,1119.00,-11.00,0.00,0.00
50%,2013.0,7.00,16.00,4.00,12892.0,12892.00,1320.00,-1.00,0.0,1529.00,-3.00,0.00,0.00
75%,2013.0,9.00,23.00,6.00,14057.0,14057.00,1725.00,9.00,0.0,1918.00,10.00,0.00,0.00
max,2013.0,10.00,31.00,7.00,15376.0,15376.00,2359.00,1425.00,1.0,2359.00,1440.00,1.00,1.00


First, we will proceed to identify any null values.

In [17]:
df_flights.isnull().sum().reset_index().sort_values(by=0,ascending=False).head()

,index,0
15,DepDel15,2761
0,Year,0
1,Month,0
18,ArrDel15,0
17,ArrDelay,0


In [27]:
df_flights.isnull().mean().round(4).mul(100).sort_values(ascending=False).head()

DepDel15       1.02
Cancelled      0.00
OriginState    0.00
Month          0.00
DayofMonth     0.00
dtype: float64

DepDel15 has 2761 missing values which corresponds to 1.02% of the total percetage of this column.

Since this is our only column with missing values, we will evaluate it to see the most appropriate way to replace information to it.

In [39]:
df_flights['DepDel15'].value_counts()

0.0    215038
1.0     54141
Name: DepDel15, dtype: int64

In [43]:
df_flights['DepDel15'].fillna(df_flights['DepDel15'].min(),inplace=True)

In [36]:
duplicados =  df_flights.duplicated().value_counts()
print(duplicados)

False    271940
dtype: int64


Our dataset does not show duplicated values.